In [21]:
!nvidia-smi

Wed Dec 24 14:39:16 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [22]:
%%writefile cuda_sorts.cu

#include <cuda_runtime.h>              // Основные функции CUDA
#include <device_launch_parameters.h>  // Параметры запуска kernel

#include <iostream>    // Ввод-вывод
#include <vector>      // Контейнер vector
#include <algorithm>   // std::sort
#include <chrono>      // Замер времени
#include <random>      // Генерация случайных чисел

#define THREADS 256     // Количество потоков в CUDA-блоке

// ------------------------------------------------------------
// Device-функция обмена элементов
// Используется внутри CUDA kernel, std::swap применять нельзя
// ------------------------------------------------------------
__device__ __forceinline__ void deviceSwap(int &a, int &b) {
    int tmp = a;
    a = b;
    b = tmp;
}

// ------------------------------------------------------------
// Bitonic sort внутри одного CUDA-блока
// Каждый блок сортирует свой подмассив в shared memory
// ------------------------------------------------------------
__global__ void bitonicSortKernel(int* data, int chunkSize) {
    // Shared memory для текущего блока
    extern __shared__ int s[];

    // Индекс потока внутри блока
    int tid = threadIdx.x;
    // Глобальный индекс элемента массива
    int idx = blockIdx.x * chunkSize + tid;

    // Загружаем данные из глобальной памяти в shared memory
    s[tid] = data[idx];
    __syncthreads(); // Ждём, пока все потоки загрузят данные

    // Основные этапы bitonic sort
    for (int k = 2; k <= chunkSize; k <<= 1) {
        for (int j = k >> 1; j > 0; j >>= 1) {
            int ixj = tid ^ j;

            if (ixj > tid) {
                // Сортировка по возрастанию
                if ((tid & k) == 0 && s[tid] > s[ixj])
                    deviceSwap(s[tid], s[ixj]);

                // Сортировка по убыванию
                if ((tid & k) != 0 && s[tid] < s[ixj])
                    deviceSwap(s[tid], s[ixj]);
            }
            __syncthreads(); // Синхронизация потоков
        }
    }

    // Записываем отсортированные данные обратно в глобальную память
    data[idx] = s[tid];
}

// ------------------------------------------------------------
// Kernel для параллельного слияния подмассивов
// Каждая нить сливает одну пару отсортированных блоков
// ------------------------------------------------------------
__global__ void mergeKernel(int* input, int* output, int n, int width) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    // Левая граница текущей пары подмассивов
    int left = tid * 2 * width;

    if (left >= n) return;

    int mid   = min(left + width, n);
    int right = min(left + 2 * width, n);

    int i = left;   // Указатель на левый подмассив
    int j = mid;    // Указатель на правый подмассив
    int k = left;   // Индекс записи в выходной массив

    // Слияние двух отсортированных подмассивов
    while (i < mid && j < right)
        output[k++] = (input[i] < input[j]) ? input[i++] : input[j++];

    // Копирование оставшихся элементов
    while (i < mid)   output[k++] = input[i++];
    while (j < right) output[k++] = input[j++];
}

// ------------------------------------------------------------
// Заполнение массива случайными числами
// ------------------------------------------------------------
void fillRandom(std::vector<int>& a) {
    std::mt19937 gen(42); // Фиксированный seed
    std::uniform_int_distribution<> dist(0, 1000000);
    for (auto& x : a)
        x = dist(gen);
}

// ------------------------------------------------------------
// Последовательная сортировка на CPU
// ------------------------------------------------------------
void sortCPU(std::vector<int>& a) {
    std::sort(a.begin(), a.end());
}

// ------------------------------------------------------------
// Главная функция
// ------------------------------------------------------------
int main() {
    // Размеры массивов для эксперимента
    std::vector<int> sizes = {10000, 100000};

    for (int n : sizes) {
        std::cout << "\nРазмер массива: " << n << std::endl;

        // Исходные данные
        std::vector<int> h_data(n);
        fillRandom(h_data);

        // ---------------- CPU ----------------
        auto cpuData = h_data;
        auto cpuStart = std::chrono::high_resolution_clock::now();
        sortCPU(cpuData);
        auto cpuEnd = std::chrono::high_resolution_clock::now();

        double cpuTime =
            std::chrono::duration<double, std::milli>(cpuEnd - cpuStart).count();

        std::cout << "CPU сортировка: " << cpuTime << " ms" << std::endl;

        // ---------------- GPU ----------------
        int* d_data;
        int* d_temp;

        // Выделение памяти на GPU
        cudaMalloc(&d_data, n * sizeof(int));
        cudaMalloc(&d_temp, n * sizeof(int));

        // Копирование данных с CPU на GPU
        cudaMemcpy(d_data, h_data.data(),
                   n * sizeof(int), cudaMemcpyHostToDevice);

        // CUDA таймеры
        cudaEvent_t start, stop;
        cudaEventCreate(&start);
        cudaEventCreate(&stop);

        int chunkSize = 1024;     // Размер подмассива
        int blocks = n / chunkSize;

        cudaEventRecord(start);

        // Локальная сортировка подмассивов
        bitonicSortKernel<<<blocks, chunkSize,
                            chunkSize * sizeof(int)>>>(d_data, chunkSize);

        // Поэтапное слияние
        for (int width = chunkSize; width < n; width <<= 1) {
            int mergeBlocks = (n + 2 * width - 1) / (2 * width);
            mergeKernel<<<mergeBlocks, THREADS>>>(d_data, d_temp, n, width);
            std::swap(d_data, d_temp);
        }

        cudaEventRecord(stop);
        cudaEventSynchronize(stop);

        float gpuTime = 0;
        cudaEventElapsedTime(&gpuTime, start, stop);

        std::cout << "GPU merge sort: " << gpuTime << " ms" << std::endl;

        // Освобождение памяти
        cudaFree(d_data);
        cudaFree(d_temp);
    }

    return 0;
}


Overwriting cuda_sorts.cu


In [23]:
!ls

cuda_sorts  cuda_sorts.cu  practical_work_3.cu	sample_data


In [24]:
!nvcc cuda_sorts.cu -o cuda_sorts

In [25]:
!./cuda_sorts


Размер массива: 10000
CPU сортировка: 4.57901 ms
GPU merge sort: 7.56461 ms

Размер массива: 100000
CPU сортировка: 35.6195 ms
GPU merge sort: 0.002432 ms
